# Punto 3 - Final - Miguel Rios Tangarife 

## Utilizar la matriz tf-idf y crear un algoritmo gradiente descendiente para clasificar los textos. Código debe ser genérico, es decir debe funcionar para textos con mas palabras


error =  y *log (1/(1+ e(-g)) + (1-y)*log (1 - 1/(1+ e(-g)) donde g = b1 palabra1+ b2 tf-idf(palabra2)+b3 tf-idf(palabra3) ….+ bn tf-idf(palabran)+b0.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [6]:
encoding = 'latin1'
ruta_archivo_csv = 'data/datoss.csv'
datosFinal = pd.read_csv(ruta_archivo_csv, encoding=encoding)

# Pasamos todos los datos a minúsculas
datosFinal['Texto'] = datosFinal['Texto'].str.lower()

#Definimos stop_words
stop_words = [
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se',
    'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'es',
    'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'fue', 'ha'
]

# Filtrar palabras no deseadas
datosFinal['Texto'] = datosFinal['Texto'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Convertir todo el texto a minúsculas después de la limpieza
datosFinalClear = datosFinal.copy()
datosFinalClear['Texto'] = datosFinalClear['Texto'].str.lower()

In [7]:
vocabulario = set(' '.join(datosFinalClear['Texto']).split())


tf_matrix = np.zeros((len(datosFinalClear), len(vocabulario)))
for i, doc in enumerate(datosFinalClear['Texto']):
    for j, word in enumerate(vocabulario):
        tf_matrix[i, j] = doc.split().count(word)
idf_vector = np.log(len(datosFinalClear) / np.sum(tf_matrix > 0, axis=0))
tfidf_matrix = tf_matrix * idf_vector

tfidf_df = pd.DataFrame(tfidf_matrix, columns=list(vocabulario))

In [16]:
X = np.column_stack((np.ones(len(datosFinalClear)), tfidf_matrix))
y = datosFinal['Sentimiento'].map({'Negativo': 0, 'Positivo': 1}).values

# Inicializamos en ceros los parámetros
params = np.zeros(X.shape[1])

def predict(X, params):
    return 1 / (1 + np.exp(-np.dot(X, params)))

def loss(y, y_pred):
    return -np.mean((y * np.log(y_pred + 1e-15)) + ((1 - y) * np.log(1 - y_pred + 1e-15)))

def gradient_descent(X, y, params, learning_rate, epochs, batch_size):
    m = len(y)
    for epoch in range(epochs):
        for batch_start in range(0, m, batch_size):
            batch_end = batch_start + batch_size
            X_batch = X[batch_start:batch_end]
            y_batch = y[batch_start:batch_end]

            y_pred = predict(X_batch, params)
            error = loss(y_batch, y_pred)

            # Calcular los gradientes
            gradient_b0 = np.sum(y_pred - y_batch) / batch_size
            gradient_b = np.dot(X_batch.T, (y_pred - y_batch)) / batch_size

            # Actualizar los parámetros
            params -= learning_rate * gradient_b

        if epoch % 1 == 0:
            print(f'Iteracion {epoch}, Loss: {error}')
            print(f'b1: {params[0]}, b2: {params[1]}, b3: {params[2]}, b4: {params[3]}')
            print('')

    return params

learning_rate = 0.5
epochs = 15
batch_size = 3

trained_params = gradient_descent(X, y, params, learning_rate, epochs, batch_size)

Iteracion 0, Loss: 0.8126192204036744
b1: 0.38015391502140994, b2: 0.28115425687979856, b3: -0.238950729266818, b4: 0.2450703816987542

Iteracion 1, Loss: 0.080062676814599
b1: 0.4812937199252677, b2: 0.3601150843598484, b3: -0.31101090555601785, b4: 0.14615305901827413

Iteracion 2, Loss: 0.032059635745666046
b1: 0.558037801596017, b2: 0.39431467437451007, b3: -0.35578437353106634, b4: 0.11750678527805022

Iteracion 3, Loss: 0.021679935377612654
b1: 0.6162398458649393, b2: 0.41739975787467654, b3: -0.3888964797833553, b4: 0.10325009323431877

Iteracion 4, Loss: 0.01680267234678754
b1: 0.6625482974403509, b2: 0.4349966243297303, b3: -0.41529601617360756, b4: 0.09426548524971487

Iteracion 5, Loss: 0.013945553654500373
b1: 0.7008924876661707, b2: 0.4492808196290853, b3: -0.4372872218908391, b4: 0.08792111471745107

Iteracion 6, Loss: 0.012059459698902026
b1: 0.7335759515494734, b2: 0.4613373314875229, b3: -0.45614950058759246, b4: 0.08312823799465249

Iteracion 7, Loss: 0.01071408049568

Observaciones dados los valores de b1,b2,b3 y b4
- b1 en un la iteracion 1 tiene un valor de 0.3801, el valor va en aumento en las siguientes iteraciones
  se puede ver que b1 tiende más a valores positivos con cada iteración.
  
- Existe una tendencia en los valores de b2,b3 y b4 a aumentar respectivamente 